In [3]:
import pandas as pd
import folium

# Load your data
data = pd.read_csv('Retail_Food_Stores_20241010.csv')

# List of target zip codes
target_zipcodes = ['10036', '10001', '10012', '11101', '11206', '11211', '11249']

# Filter for stores in the target zip codes
filtered_data = data[data['Zip Code'].astype(str).isin(target_zipcodes)]

# Count stores in each zip code
store_counts = filtered_data['Zip Code'].value_counts()
print("Store counts by zip code:")
print(store_counts)

# Extract latitude and longitude from 'Georeference' column
filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)

# Create a map centered around the NYC area
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add markers for each store
for _, row in filtered_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['DBA Name']
    ).add_to(nyc_map)

# Display the map
nyc_map.save("nyc_retail_map.html")


Store counts by zip code:
Zip Code
11211    150
11206    124
11101     96
10001     46
11249     43
10012     38
10036     37
Name: count, dtype: int64


C:\Users\zheng\AppData\Local\Temp\ipykernel_5160\2629115881.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)
C:\Users\zheng\AppData\Local\Temp\ipykernel_5160\2629115881.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)
